<a href="https://colab.research.google.com/github/AnthoMant/Creuse-Movie-Project/blob/main/Base_de_donn%C3%A9e_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#data viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px 
import plotly.graph_objects as go

#preprocess
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

#models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

#metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score, precision_score,recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

#autres:
import time
import math as m
from scipy.spatial import distance
from google.colab import drive

##Importation des bases de données

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_noFilter=pd.read_csv('/content/drive/MyDrive/df_final.csv')
title_crew = pd.read_csv('https://datasets.imdbws.com/title.crew.tsv.gz', sep='\t')
title_principals = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz', sep='\t')

In [7]:
df_noFilter

,Unnamed: 0,tconst,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,decade
0,5013,tt0005076,Charley's Aunt,1925,80,Comedy,6.7,49,1920
1,8810,tt0008933,Die Brüder Karamasoff,1920,70,Drama,6.5,12,1920
2,9639,tt0009771,A víg özvegy,1920,70,Musical,4.5,15,1920
3,9922,tt0010058,The Deadlier Sex,1920,60,"Comedy,Drama",6.3,11,1920
4,10186,tt0010323,Das Cabinet des Dr. Caligari,1920,76,"Fantasy,Horror,Mystery",8.1,58202,1920
...,...,...,...,...,...,...,...,...,...
213133,7823839,tt9916160,Drømmeland,2019,72,Documentary,6.5,41,2010
213134,7823853,tt9916190,Safeguard,2020,90,"Action,Adventure,Thriller",3.4,140,2020
213135,7823892,tt9916270,Il talento del calabrone,2020,84,Thriller,5.7,1014,2020
213136,7823937,tt9916362,Akelarre,2020,92,"Adventure,Drama,History",6.3,2937,2020


In [5]:
title_crew

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N
...,...,...,...
7872538,tt9916848,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7872539,tt9916850,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7872540,tt9916852,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7872541,tt9916856,nm10538645,nm6951431


In [12]:
title_principals

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
...,...,...,...,...,...,...
44525535,tt9916880,5,nm0996406,director,principal director,\N
44525536,tt9916880,6,nm1482639,writer,\N,\N
44525537,tt9916880,7,nm2586970,writer,books,\N
44525538,tt9916880,8,nm1594058,producer,producer,\N


## Ajout des "Actor/Actress" à la base de donnée

In [13]:
title_actor=title_principals[(title_principals["category"].str.contains("actor"))|(title_principals["category"].str.contains("actress"))]

In [ ]:
title_actor.drop(["ordering","category","job","characters"], axis=1 , inplace=True)

In [16]:
title_actor

,tconst,nconst
11,tt0000005,nm0443482
12,tt0000005,nm0653042
16,tt0000007,nm0179163
17,tt0000007,nm0183947
21,tt0000008,nm0653028
...,...,...
44525531,tt9916880,nm1483166
44525532,tt9916880,nm0254176
44525533,tt9916880,nm0286175
44525534,tt9916880,nm10535738


In [17]:
df_noFilter=df_noFilter.merge(title_actor, how='inner', on='tconst')

In [ ]:
# concatenate the string
df_noFilter['nconst'] = df_noFilter.groupby(['tconst'])['nconst'].transform(lambda x : ','.join(x))
  
# drop duplicate data
df_noFilter = df_noFilter.drop_duplicates()   
  

##Ajout des "directors" à la base de donnée

In [20]:
df_noFilter=df_noFilter.merge(title_crew, how='inner', on='tconst')

##Suppresion des colonnes inutiles

In [22]:
df_noFilter.drop("Unnamed: 0",axis=1,inplace=True)

In [23]:
df_noFilter.drop("writers",axis=1,inplace=True)

##Modification du nom de la colonne "nconst"

In [26]:
df_noFilter.columns = ["tconst","originalTitle","startYear","runtimeMinutes","genres","averageRating","numVotes","decade","actors","directors"]

In [27]:
df_noFilter

,tconst,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,decade,actors,directors
0,tt0005076,Charley's Aunt,1925,80,Comedy,6.7,49,1920,"nm0152260,nm0788281,nm0656200,nm0514707",nm0796657
1,tt0008933,Die Brüder Karamasoff,1920,70,Drama,6.5,12,1920,"nm0466776,nm0324553,nm0417837,nm0858164","nm0296193,nm0118266"
2,tt0009771,A víg özvegy,1920,70,Musical,4.5,15,1920,"nm0889511,nm2381950,nm2395932,nm0489988",nm0002031
3,tt0010058,The Deadlier Sex,1920,60,"Comedy,Drama",6.3,11,1920,"nm0801193,nm0842239,nm0356228,nm0481764,nm0358029",nm0861336
4,tt0010323,Das Cabinet des Dr. Caligari,1920,76,"Fantasy,Horror,Mystery",8.1,58202,1920,"nm0470328,nm0891998,nm0270415,nm0196820",nm0927468
...,...,...,...,...,...,...,...,...,...,...
191455,tt9915872,My Girlfriend is a Wizard,2019,97,"Comedy,Drama,Fantasy",6.8,5,2010,"nm0755725,nm4392634",nm8063415
191456,tt9916190,Safeguard,2020,90,"Action,Adventure,Thriller",3.4,140,2020,"nm0302466,nm2009056,nm2018573,nm2952127",nm7308376
191457,tt9916270,Il talento del calabrone,2020,84,Thriller,5.7,1014,2020,"nm0144812,nm3080119,nm2063290,nm9428255",nm1480867
191458,tt9916362,Akelarre,2020,92,"Adventure,Drama,History",6.3,2937,2020,"nm3766704,nm0107165,nm0266723,nm10678594",nm1893148


##Verification des /N

In [29]:
df_noFilter[df_noFilter["directors"]=="\\N"]

,tconst,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,decade,actors,directors
437,tt0014228,Luck,1923,70,"Action,Comedy,Sport",6.9,9,1920,"nm0614879,nm0385686,nm0581499,nm0249187,nm0106785",\N
11208,tt0037854,Kong Håkon VIIs regjeringsjubileum 1945,1945,94,"Documentary,History",6.6,8,1940,nm0085098,\N
14462,tt0043289,Antony and Cleopatra,1951,169,"Drama,History,Romance",4.9,11,1950,"nm0504829,nm0817249",\N
16345,tt0046222,A Queen Is Crowned,1953,82,"Documentary,History",7.3,98,1950,nm0000059,\N
22449,tt0055215,Naked Terror,1961,74,Documentary,7.2,10,1960,nm0001637,\N
...,...,...,...,...,...,...,...,...,...,...
190040,tt9387300,Girls Love,2016,61,Romance,6.7,19,2010,"nm10305176,nm10305177",\N
190192,tt9448868,BTS World Tour: Love Yourself in Seoul,2018,112,"Documentary,Music",8.9,858,2010,"nm8506459,nm8062274,nm8506457,nm8506461,nm6534...",\N
190224,tt9461496,Anaganaga O Premakatha,2018,125,Drama,5.8,56,2010,"nm10337629,nm9661819",\N
190283,tt9486348,Cinema Enthusiast,2000,95,"Action,Comedy,Drama",4.7,32,2000,"nm1193573,nm1382197,nm1220642,nm1232442,nm1717...",\N


Pour le bon fonctionnement de notre modeles, nous décidons de supprimer les /N

In [30]:
df_noFilter.drop(df_noFilter.loc[df_noFilter["directors"]=='\\N'].index, inplace=True)

##Exportation de la base de donnée

In [31]:
df_noFilter.to_csv('df_final_projet2.csv')
!cp df_final_projet2.csv "drive/My Drive/"